In [5]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on 2024-07-18 09:44:51 Thursday

@author: Nikhil Kapila
"""

'\nCreated on 2024-07-18 09:44:51 Thursday\n\n@author: Nikhil Kapila\n'

In [6]:
import os, re, io, contextlib, glob
import pandas as pd
# from metaflow import Flow
from metaflow import Runner, Flow, Task
os.getcwd()

'/Users/nikhilkapila/Developer/lstm-training'

In [7]:
ls

README.md                    metaflow_NBRunner.ipynb
__pycache__/                 metaflow_models_30hrs/
bgd_flow.py                  metaflow_models_360hrs/
classes/                     metaflow_models_360hrs copy/
datasets/


In [8]:
print(pd.Timestamp.now().strftime('_%Y%m%d_%H%M'))

_20240722_2252


In [9]:
datasets = 'datasets/building_genome_dataset/data_subsets'
datasets = [f for f in os.listdir(datasets) 
              if os.path.isfile(os.path.join(datasets, f)) and f != '.DS_Store']
print(datasets)

['bgd_Office_5001_10000.csv', 'bgd_Retail_100001_200000.csv', 'bgd_entertainment_public_assembly_25001_50000.csv', 'bgd_Parking_50001_100000.csv', 'bgd_Office_200001_500000.csv', 'bgd_entertainment_public_assembly_5001_10000.csv', 'bgd_Religious_worship_5001_10000.csv', 'bgd_Education_100001_200000.csv', 'bgd_Food_sales_service_10001_25000.csv', 'bgd_Education_1001_5000.csv', 'bgd_Retail_25001_50000.csv', 'bgd_entertainment_public_assembly_200001_500000.csv', 'bgd_entertainment_public_assembly_50001_100000.csv', 'bgd_warehouse_storage_10001_25000.csv', 'bgd_Healthcare_10001_25000.csv', 'bgd_warehouse_storage_200001_500000.csv', 'bgd_lodging_residential_1001_5000.csv', 'bgd_Healthcare_200001_500000.csv', 'bgd_Parking_200001_500000.csv', 'bgd_Office_25001_50000.csv', 'bgd_Other_5001_10000.csv', 'bgd_lodging_residential_100001_200000.csv', 'bgd_Other_1001_5000.csv', 'bgd_Education_10001_25000.csv', 'bgd_Public_services_50001_100000.csv', 'bgd_Public_services_10001_25000.csv', 'bgd_Public_

In [10]:
dataset = 'bgd_Office_5001_10000.csv'
pattern = r"bgd_(?P<type>\w+)_(?P<size>\d+_\d+).csv"
match = re.search(pattern, dataset)
print(pattern)
print(dataset)
print(match.group('type'))
print(match.group('size'))

bgd_(?P<type>\w+)_(?P<size>\d+_\d+).csv
bgd_Office_5001_10000.csv
Office
5001_10000


In [11]:
# types = ['Office', 'Retail', 'entertainment_public_assembly', 'Parking', 'Religious_worship', 'Education', 'Food_sales_service', 'warehouse_storage', 'Healthcare', 'lodging_residential', 'Other', 'Public_services', 'Utility', 'technology_science']
# sizes = ['5001_10000', '100001_200000', '25001_50000', '50001_100000', '200001_500000', '10001_25000', '1001_5000']

In [12]:
path_dir = f'{dataset[:-4]}_LSTM'
path_dir

'bgd_Office_5001_10000_LSTM'

In [13]:
def check_if_run_is_completed(model_name:str, flow_name:str='BGD_LinearFlow'):
    flow = Flow(flow_name)
    for run in flow.runs():
        if run.successful and run.data.model_name == model_name:
            return run
    return None

In [14]:
old_run = check_if_run_is_completed(model_name='bgd_education_1001_5000_LSTM')
print(old_run)

old_run_1 = check_if_run_is_completed(model_name='hahah')
print(old_run_1)

Run('BGD_LinearFlow/7')
None


In [15]:
num_of_models = 0

for dataset in datasets:
    match = re.search(pattern, dataset)
    type = match.group('type')
    size = match.group('size')
    if type=='Office' or type=='Education' or type=='lodging_residential':
        num_of_models = num_of_models + 1

num_of_models

19

In [16]:
def check_if_run_is_completed_dir_360(model_name:str, flow_name:str='BGD_LinearFlow'):
    flow = Flow(flow_name)
    for run in flow.runs():
        if run.finished == True:
            if run.data.dir == 'metaflow_models_360hrs':
                return run
    return None

In [17]:
dataset[:-4]+'_LSTM'

'bgd_Healthcare_5001_10000_LSTM'

In [18]:
folders = glob.glob('metaflow_models_360hrs/bgd_*_*_*_LSTM')
folders

['metaflow_models_360hrs/bgd_office_200001_500000_LSTM',
 'metaflow_models_360hrs/bgd_office_5001_10000_LSTM']

In [19]:
save_dir = 'metaflow_models_360hrs/'
model_name = 'bgd_office_200001_500000_LSTM'

if f'{save_dir}{model_name}' in folders: 
    print(True)
    print(f'{save_dir}{model_name}')
else: print('urmum')

True
metaflow_models_360hrs/bgd_office_200001_500000_LSTM


In [20]:
# building_types, size_ranges = [], []
pattern = r"bgd_(?P<type>\w+)_(?P<size>\d+_\d+).csv"
lookback = 360
save_dir = 'metaflow_models_360hrs/'
folders = glob.glob(f'{save_dir}bgd_*_*_*_LSTM')

for dataset in datasets:
    skip_flag = 10
    match = re.search(pattern, dataset)
    type = match.group('type')
    size = match.group('size')
    if type=='Office' or type=='Education' or type=='lodging_residential':
        # if type not in building_types: building_types.append(type)
        # if size not in size_ranges: size_ranges.append(size)
        flow_tag = f"{type}_{size}_{pd.Timestamp.now().strftime('%Y%m_%H%M')}"
        fpath = f'/Users/nikhilkapila/Developer/lstm-training/datasets/building_genome_dataset/data_subsets/{dataset}'
        model_name = f'{dataset[:-4]}_LSTM'
        folder_check = f'{save_dir}{model_name}'
        # fpath = f'/datasets/building_genome_dataset/data_subsets/{dataset}'
        outputfilename = f'/Users/nikhilkapila/Developer/lstm-training/{save_dir}{dataset[:-4]}_LSTM/output.txt'

        for folder in folders:
            if folder_check.lower() == folder.lower(): skip_flag = 20

        if skip_flag == 20:
            print(f"Skipping execution because a successful run with model_name '{model_name}' already exists.")
        elif skip_flag == 10:
            print(f'Starting metaflow run for {dataset}')
            f = io.StringIO()
            with contextlib.redirect_stdout(f):
                with Runner('bgd_flow.py', pylint=False).run(tags=[flow_tag], filepath=fpath, lookback=lookback, dir=save_dir) as running:
                    if running.status == 'failed':
                        print(f'❌ {running.run} failed:')
                    elif running.status == 'successful':
                        print(f'✅ {running.run} succeeded:')
                    print(f'-- stdout --\n{running.stdout}')
                    print(f'-- stderr --\n{running.stderr}')
                            
            with open(outputfilename, 'w') as file:
                file.write(f.getvalue())
            print(f'Output saved as {outputfilename}')
            f.truncate(0)
            f.seek(0)

Skipping execution because a successful run with model_name 'bgd_Office_5001_10000_LSTM' already exists.
Skipping execution because a successful run with model_name 'bgd_Office_200001_500000_LSTM' already exists.
Starting metaflow run for bgd_Education_100001_200000.csv
Output saved as /Users/nikhilkapila/Developer/lstm-training/metaflow_models_360hrs/bgd_Education_100001_200000_LSTM/output.txt
Starting metaflow run for bgd_Education_1001_5000.csv
Output saved as /Users/nikhilkapila/Developer/lstm-training/metaflow_models_360hrs/bgd_Education_1001_5000_LSTM/output.txt
Starting metaflow run for bgd_lodging_residential_1001_5000.csv
Output saved as /Users/nikhilkapila/Developer/lstm-training/metaflow_models_360hrs/bgd_lodging_residential_1001_5000_LSTM/output.txt
Starting metaflow run for bgd_Office_25001_50000.csv
Output saved as /Users/nikhilkapila/Developer/lstm-training/metaflow_models_360hrs/bgd_Office_25001_50000_LSTM/output.txt
Starting metaflow run for bgd_lodging_residential_100